In [1]:
import pandas as pd

In [2]:
d1 = pd.read_csv('data/dm_01_all.csv').set_index(['times']).sort_index()
d2 = pd.read_csv('data/dm_02_all.csv').set_index(['times']).sort_index()

In [3]:
len(d2.loc[0.2].sort_values('senders'))

50

In [4]:
s3_1 = d1.loc[0.3].sort_values('senders')
s3_2 = d2.loc[0.3].sort_values('senders')

In [5]:
eq_next = s3_2.iloc[1:].senders == s3_2.iloc[:-1].senders

In [6]:
s3_2.iloc[1:].loc[eq_next].senders

times
0.3     1
0.3     6
0.3     8
0.3    13
0.3    15
0.3    22
0.3    24
0.3    27
0.3    31
0.3    33
0.3    40
0.3    41
0.3    42
0.3    44
0.3    46
0.3    47
0.3    49
Name: senders, dtype: int64

In [7]:
bad_sources = [10, 49, 41, 41, 9, 44, 19, 46, 9, 25, 11, 33, 46, 37,
               36, 27, 45, 29, 15, 27, 21, 50, 27, 38, 3, 5, 38, 3,
               41, 49, 42, 37, 36, 45, 5, 3, 21, 29, 9, 30, 34, 40,
               35, 44, 41, 48, 44, 27, 36, 47]

In [8]:
good_sources = [45, 50, 37, 13, 47, 29, 9, 46, 15, 10, 15, 38, 34,
                29, 47, 45, 14, 23, 35, 1, 44, 3, 20, 46, 46, 13, 3,
                49, 3, 48, 5, 9, 15, 28, 30, 25, 40, 30, 16, 3, 40,
                40, 24, 40, 40, 17, 50, 32, 43, 42 ]

In [9]:
tgts = list(range(1, 51))

In [10]:
bad = pd.DataFrame(zip(bad_sources, tgts), columns=['Src', 'Tgt'])
good = pd.DataFrame(zip(good_sources, tgts), columns=['Src', 'Tgt'])

In [11]:
def add_vp(conns):
    conns['Sv'] = conns.Src % 4
    conns['Tv'] = conns.Tgt % 4

    conns['Sr'] = conns.Sv % 2
    conns['Tr'] = conns.Tv % 2

    conns['St'] = (conns.Sv // 2) % 2
    conns['Tt'] = (conns.Tv // 2) % 2
    
    return conns

In [12]:
bad = add_vp(bad)
good = add_vp(good)

In [13]:
for fr in range(2):
    for tr in range(2):
        print(fr, tr, sum((bad.Sr == fr) & (bad.Tr == tr)))

0 0 8
0 1 9
1 0 17
1 1 16


In [14]:
for fr in range(2):
    for tr in range(2):
        print(fr, tr, sum((good.Sr == fr) & (good.Tr == tr)))

0 0 12
0 1 12
1 0 13
1 1 13


In [15]:
len(set(good_sources))

32

In [16]:
len(set(bad_sources))

28

In [17]:
def stt(ct, nvp, nmpi):
    ct = ct.copy()
    ct['Tr'] = ct.apply(lambda r: (r.Tgt % nvp) % nmpi, axis=1)
    ct = pd.DataFrame(list(ct.groupby(['Src', 'Tr']).Tgt.count().index.values), columns=['sn', 'tr'])
    ct['sr'] = ct.apply(lambda r: (r.sn % nvp) % nmpi, axis=1)
    return ct.groupby(['sr', 'tr']).count()

In [18]:
stt(bad, 4, 2)

sn
sr tr    
0  0    7
   1    7
1  0   11
   1   11

In [19]:
stt(good, 4, 2)

sn
sr tr    
0  0   10
   1   10
1  0   10
   1    9

In [20]:
stt(good, 4, 1)

,,sn
sr,tr,
0,0,32


In [21]:
stt(bad, 4, 1)

,,sn
sr,tr,
0,0,28


In [22]:
bad.loc[bad.Sr==0].groupby(['Src', 'Tr']).Tgt.count()

Src  Tr
10   1     1
30   0     1
34   1     1
36   1     3
38   0     1
     1     1
40   0     1
42   1     1
44   0     2
     1     1
46   0     1
     1     1
48   0     1
50   0     1
Name: Tgt, dtype: int64

In [23]:
lg = """sr called: r0 t0 s4 m1
sr called: r0 t1 s2 m1
sr called: r0 t0 s8 m1
sr called: r0 t1 s6 m1
sr called: r0 t0 s12 m1
sr called: r0 t1 s10 m1
sr called: r0 t0 s16 m1
r0 t1 s10 m1 l2 tr1
sr called: r0 t0 s20 m1
r0 t1 s10 m1 l2 tr1
sr called: r0 t0 s24 m1
sr called: r0 t1 s14 m1
sr called: r0 t0 s28 m1
sr called: r0 t1 s18 m1
sr called: r0 t0 s32 m1
sr called: r0 t1 s22 m1
sr called: r0 t0 s36 m1
sr called: r0 t1 s26 m1
r0 t0 s36 m1 l5 tr1
sr called: r0 t1 s30 m1
r0 t0 s36 m1 l5 tr1
r0 t1 s30 m1 l2 tr0
sr called: r0 t0 s40 m1
r0 t1 s30 m1 l2 tr0
r0 t0 s40 m1 l12 tr0
sr called: r0 t1 s34 m1
r0 t0 s40 m1 l12 tr0
r0 t1 s34 m1 l4 tr1
sr called: r0 t0 s44 m1
r0 t1 s34 m1 l4 tr1
r0 t0 s44 m1 l7 tr0
sr called: r0 t1 s38 m1
r0 t0 s44 m1 l17 tr1
r0 t1 s38 m1 l5 tr0
r0 t0 s44 m1 l7 tr0
r0 t1 s38 m1 l15 tr1
r0 t0 s44 m1 l17 tr1
r0 t1 s38 m1 l5 tr0
sr called: r0 t0 s48 m1
r0 t1 s38 m1 l15 tr1
r0 t0 s48 m1 l15 tr0
sr called: r0 t1 s42 m1
r0 t0 s48 m1 l15 tr0
r0 t1 s42 m1 l16 tr1
r0 t1 s42 m1 l16 tr1
sr called: r0 t1 s46 m1
r0 t1 s46 m1 l8 tr0
r0 t1 s46 m1 l8 tr1
r0 t1 s46 m1 l8 tr0
r0 t1 s46 m1 l8 tr1
sr called: r0 t1 s50 m1
r0 t1 s50 m1 l17 tr0
r0 t1 s50 m1 l17 tr0"""


In [25]:
sorted([l for l in lg.splitlines() if not l.startswith('sr')])

['r0 t0 s36 m1 l5 tr1',
 'r0 t0 s36 m1 l5 tr1',
 'r0 t0 s40 m1 l12 tr0',
 'r0 t0 s40 m1 l12 tr0',
 'r0 t0 s44 m1 l17 tr1',
 'r0 t0 s44 m1 l17 tr1',
 'r0 t0 s44 m1 l7 tr0',
 'r0 t0 s44 m1 l7 tr0',
 'r0 t0 s48 m1 l15 tr0',
 'r0 t0 s48 m1 l15 tr0',
 'r0 t1 s10 m1 l2 tr1',
 'r0 t1 s10 m1 l2 tr1',
 'r0 t1 s30 m1 l2 tr0',
 'r0 t1 s30 m1 l2 tr0',
 'r0 t1 s34 m1 l4 tr1',
 'r0 t1 s34 m1 l4 tr1',
 'r0 t1 s38 m1 l15 tr1',
 'r0 t1 s38 m1 l15 tr1',
 'r0 t1 s38 m1 l5 tr0',
 'r0 t1 s38 m1 l5 tr0',
 'r0 t1 s42 m1 l16 tr1',
 'r0 t1 s42 m1 l16 tr1',
 'r0 t1 s46 m1 l8 tr0',
 'r0 t1 s46 m1 l8 tr0',
 'r0 t1 s46 m1 l8 tr1',
 'r0 t1 s46 m1 l8 tr1',
 'r0 t1 s50 m1 l17 tr0',
 'r0 t1 s50 m1 l17 tr0']

In [63]:
def source_table(src, nvp, nr):
    stab = [[[] for _ in range(nvp//nr)] for _ in range(nr)]
    for ix, s in enumerate(src):
        t = ix + 1
            
        sv, tv = s % nvp, t % nvp
        sr, tr = sv % nr, tv % nr
        st, tt = sv // nr, tv // nr

        stab[tr][tt].append((s, t, tt))
    for l in stab:
        for ll in l:
            ll.sort()
    return stab

In [69]:
def compress_one_thread(ttab):
    ctab = []
    lcid = -1
    last_gid = -1
    for s, t, tt in ttab:
        lcid += 1
        if s != last_gid:
            last_gid = s
            ctab.append((s, (tt, lcid)))
    return ctab

def compress_sources(stab):
    return [[compress_one_thread(ttab) for ttab in rtab] for rtab in stab]

In [85]:
def fc_per_rank(ctr):
    cmap = {}
    csd = []
    for tix, cs in enumerate(ctr):
        for src, (tt, lcid) in cs:
            assert tix == tt
            if src not in cmap:
                cmap[src] = len(csd)
                csd.append(list([] for _ in range(len(ctr))))
            six = cmap[src]
            csd[six][tt].append({'lcid': lcid, 'tt': tt})
    return cmap, csd

def final_compression(ctab):
    return [fc_per_rank(ctr) for ctr in ctab]

In [81]:
%pdb

Automatic pdb calling has been turned ON


In [86]:
stb2 = source_table(bad_sources, 4, 2)
cstb2 = compress_sources(stb2)
ftb2 = final_compression(cstb2)

In [87]:
ftb2

[({3: 0,
   27: 1,
   30: 2,
   33: 3,
   37: 4,
   38: 5,
   41: 6,
   44: 7,
   46: 8,
   5: 9,
   25: 10,
   29: 11,
   40: 12,
   45: 13,
   47: 14,
   48: 15,
   49: 16,
   50: 17},
  [[[{'lcid': 0, 'tt': 0}], []],
   [[{'lcid': 2, 'tt': 0}], []],
   [[{'lcid': 5, 'tt': 0}], []],
   [[{'lcid': 6, 'tt': 0}], []],
   [[{'lcid': 7, 'tt': 0}], [{'lcid': 4, 'tt': 1}]],
   [[{'lcid': 8, 'tt': 0}], []],
   [[{'lcid': 9, 'tt': 0}], []],
   [[{'lcid': 10, 'tt': 0}], [{'lcid': 6, 'tt': 1}]],
   [[{'lcid': 11, 'tt': 0}], []],
   [[], [{'lcid': 0, 'tt': 1}]],
   [[], [{'lcid': 1, 'tt': 1}]],
   [[], [{'lcid': 2, 'tt': 1}]],
   [[], [{'lcid': 5, 'tt': 1}]],
   [[], [{'lcid': 7, 'tt': 1}]],
   [[], [{'lcid': 8, 'tt': 1}]],
   [[], [{'lcid': 9, 'tt': 1}]],
   [[], [{'lcid': 10, 'tt': 1}]],
   [[], [{'lcid': 12, 'tt': 1}]]]),
 ({3: 0,
   9: 1,
   10: 2,
   21: 3,
   34: 4,
   36: 5,
   41: 6,
   45: 7,
   46: 8,
   5: 9,
   11: 10,
   15: 11,
   19: 12,
   27: 13,
   35: 14,
   38: 15,
   42: 16,

In [88]:
stb2

[[[(3, 28, 0),
   (3, 36, 0),
   (27, 16, 0),
   (27, 20, 0),
   (27, 48, 0),
   (30, 40, 0),
   (33, 12, 0),
   (37, 32, 0),
   (38, 24, 0),
   (41, 4, 0),
   (44, 44, 0),
   (46, 8, 0)],
  [(5, 26, 1),
   (25, 10, 1),
   (29, 18, 1),
   (29, 38, 1),
   (37, 14, 1),
   (40, 42, 1),
   (44, 6, 1),
   (45, 34, 1),
   (47, 50, 1),
   (48, 46, 1),
   (49, 2, 1),
   (49, 30, 1),
   (50, 22, 1)]],
 [[(3, 25, 0),
   (9, 5, 0),
   (9, 9, 0),
   (10, 1, 0),
   (21, 21, 0),
   (21, 37, 0),
   (34, 41, 0),
   (36, 33, 0),
   (36, 49, 0),
   (41, 29, 0),
   (41, 45, 0),
   (45, 17, 0),
   (46, 13, 0)],
  [(5, 35, 1),
   (9, 39, 1),
   (11, 11, 1),
   (15, 19, 1),
   (19, 7, 1),
   (27, 23, 1),
   (35, 43, 1),
   (36, 15, 1),
   (38, 27, 1),
   (41, 3, 1),
   (42, 31, 1),
   (44, 47, 1)]]]

In [89]:
cstb2

[[[(3, (0, 0)),
   (27, (0, 2)),
   (30, (0, 5)),
   (33, (0, 6)),
   (37, (0, 7)),
   (38, (0, 8)),
   (41, (0, 9)),
   (44, (0, 10)),
   (46, (0, 11))],
  [(5, (1, 0)),
   (25, (1, 1)),
   (29, (1, 2)),
   (37, (1, 4)),
   (40, (1, 5)),
   (44, (1, 6)),
   (45, (1, 7)),
   (47, (1, 8)),
   (48, (1, 9)),
   (49, (1, 10)),
   (50, (1, 12))]],
 [[(3, (0, 0)),
   (9, (0, 1)),
   (10, (0, 3)),
   (21, (0, 4)),
   (34, (0, 6)),
   (36, (0, 7)),
   (41, (0, 9)),
   (45, (0, 11)),
   (46, (0, 12))],
  [(5, (1, 0)),
   (9, (1, 1)),
   (11, (1, 2)),
   (15, (1, 3)),
   (19, (1, 4)),
   (27, (1, 5)),
   (35, (1, 6)),
   (36, (1, 7)),
   (38, (1, 8)),
   (41, (1, 9)),
   (42, (1, 10)),
   (44, (1, 11))]]]